In [14]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import utils
import os
from icecream import ic
import angles
from models import unetnc
from models.backwardmapper import Backwardmapper
import torch.nn as nn
#import seaborn as sns
#sns.set(color_codes=True)
data_dir= 'Dataset Preview/Inv3D preview complete V2/data/train/'
model_wc = unetnc.Estimator3d(input_nc=3, output_nc=8, num_downs=5)
model_bm = Backwardmapper()

In [10]:
path = data_dir + '00/'
ang = utils.load_anlges(path)
ang = ang.unsqueeze(0)

theta_x_gt = ang[:,0:1,:,:]
theta_y_gt = ang[:,1:2,:,:]
l_angle = model_wc.l_angle_def(theta_x_gt, theta_y_gt, theta_x_gt, theta_y_gt, 'test')
ic('Without txt mask: ',torch.mean(l_angle))

txt_msk = utils.load_txt_msk(path)
txt_msk = txt_msk.unsqueeze(0)
l_angle_msk = l_angle * txt_msk
ic('with txt msk: ', torch.mean(l_angle_msk))

ic| 'Without txt mask: ', torch.mean(l_angle): tensor(0.)
ic| 'with txt msk: ', torch.mean(l_angle_msk): tensor(0.)


('with txt msk: ', tensor(0.))

In [25]:
path = data_dir + '08/'
bm = utils.load_bm(path).unsqueeze(0).transpose(1,2).transpose(2,3)

L1_loss = nn.L1Loss(reduction='none')
l1_loss = L1_loss(bm,bm)
ic(l1_loss.mean())

uv = utils.load_uv(path).unsqueeze(0).transpose(1,2).transpose(2,3)
# angle loss noch
#ic(bm.shape)
#N, H, W, C=2 sollte es sein
angles_map = angles.calc_angles_torch(bm)
#ic(angles_map.shape)
#ic(uv.shape)
    #:param grid: Regular grid of floats (e.g. angles) with shape N, C=2, H, W
    #:param uv: UV map for given sample with shape N, H, W, C=3. Note, the y-coord channel (1) needs to be unmodified (top=1, bottom=0)!
warped_angle = angles.warp_grid_torch(angles_map, uv)
warped_angle = warped_angle.transpose(3,2).transpose(2,1)
theta_x = warped_angle[:,0:1,:,:]
theta_y = warped_angle[:,1:2,:,:]

ang = utils.load_anlges(path)
ang = ang.unsqueeze(0)
theta_x_gt = ang[:,0:1,:,:]
theta_y_gt = ang[:,1:2,:,:]
ic(theta_x.mean())
ic(theta_x_gt.mean())
ic(theta_y.mean())
ic(theta_y_gt.mean())



txt_msk = utils.load_txt_msk(path).unsqueeze(0)
l_angle = model_bm.l_angle_def(theta_x, theta_y, theta_x_gt, theta_y_gt, 'test')
ic('Without txt mask: ',l_angle.mean())
l_angle = l_angle * txt_msk
ic('with txt msk: ',l_angle.mean())

ic(l1_loss.shape, l_angle.shape)
loss = torch.mean(l1_loss.transpose(3,2).transpose(2,1) + l_angle)
ic(loss)

ic| l1_loss.mean(): tensor(0.)
ic| theta_x.mean(): tensor(-0.0009)
ic| theta_x_gt.mean(): tensor(-0.0015)
ic| theta_y.mean(): tensor(0.0096)
ic| theta_y_gt.mean(): tensor(0.0118)
ic| 'Without txt mask: ', l_angle.mean(): tensor(0.0040)
ic| 'with txt msk: ', l_angle.mean(): tensor(0.)
ic| l1_loss.shape: torch.Size([1, 256, 256, 2])
    l_angle.shape: torch.Size([1, 1, 256, 256])
ic| loss: tensor(0.)


tensor(0.)